### Import packages

In [11]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from sys import path
path.insert(0, '..')
from tuh_epilepsy.dataset import Dataset

### Read in signal data with Dataset class

In [7]:
dataset = Dataset()
# Here we get separate arrays for features and answers
features, answers = dataset()
# Transpose `features` so we have
# shape (samples, channels) for each instance
features = features.transpose((0,2,1))
print(features.shape, answers.shape)

(49963, 2500, 24) (49963,)


### Split into train and test

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    features, answers, test_size=0.1, random_state=90)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(44966, 2500, 24) (44966,) (4997, 2500, 24) (4997,)


### Build sequential model

In [21]:
input_shape = x_train[0].shape
num_classes = 2

In [22]:
model = tf.keras.models.Sequential()
layers = tf.keras.layers
model.add(layers.Conv1D(100, kernel_size=10, activation='relu', input_shape=input_shape))
model.add(layers.Conv1D(100, kernel_size=10, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(160, kernel_size=10, activation='relu'))
model.add(layers.Conv1D(160, kernel_size=10, activation='relu'))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2491, 100)         24100     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2482, 100)         100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 827, 100)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 818, 160)          160160    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 809, 160)          256160    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 160)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 160)               0         
__________

In [6]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Using training set to train model

In [7]:
import time
start = time.time()

model.fit(x_train, y_train, validation_split=0.1, epochs=50, verbose=1)

end = time.time()
round(end - start, 2), 'seconds'

Train on 40469 samples, validate on 4497 samples
Epoch 1/50
40469/40469 [==============================] - 1140s 28ms/step - loss: 6.8709 - acc: 0.5735 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 2/50
40469/40469 [==============================] - 1139s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 3/50
40469/40469 [==============================] - 1139s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 4/50
40469/40469 [==============================] - 1142s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 5/50
40469/40469 [==============================] - 1140s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 6/50
40469/40469 [==============================] - 1139s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_loss: 6.8135 - val_acc: 0.5773
Epoch 7/50
40469/40469 [==============================] - 1140s 28ms/step - loss: 6.8732 - acc: 0.5736 - val_lo

(56926.69, 'seconds')

In [9]:
# Save trained model
model.save('../tuh_epilepsy/resources/model_01.h5')
# Measure accuracy for predictions on test data
model.evaluate(x_test, y_test, verbose=2)

[6.828498750441977, 0.5763458074904547]